# Información:
- Esto es para obtener datos como:
   - Tasa de incidencia
   - Población en riesgo (casos por edad y género)
   - Clasificación clínica de los casos
- Todo es de dengue (temporalmente)
- Todos los datos se harán a nivel distrital

# Código

## Versión 1

In [ ]:
pip install --upgrade selenium webdriver-manager pandas

   ---------------------------------------- 0.0/11.0 MB ? eta -:--:--
   ---------- ----------------------------- 2.9/11.0 MB 16.8 MB/s eta 0:00:01
   ----------------------- ---------------- 6.6/11.0 MB 16.8 MB/s eta 0:00:01
   -------------------------------- ------- 8.9/11.0 MB 15.0 MB/s eta 0:00:01
   ---------------------------------------- 11.0/11.0 MB 13.5 MB/s eta 0:00:00
  Attempting uninstall: pandas
    Found existing installation: pandas 2.3.2
    Uninstalling pandas-2.3.2:
      Successfully uninstalled pandas-2.3.2
Note: you may need to restart the kernel to use updated packages.


In [7]:
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# --- Configuración de Selenium ---
options = webdriver.ChromeOptions()
# options.add_argument('--headless') # Descomenta esto al final si todo funciona
options.add_argument('--start-maximized') # Maximizar la ventana ayuda a encontrar elementos
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')

try:
    print("Configurando el controlador de Chrome...")
    service = ChromeService(ChromeDriverManager().install())
    driver = webdriver.Chrome(service=service, options=options)

    # --- Navegación y Scrapping ---
    url = "https://app7.dge.gob.pe/maps/sala_metaxenica/"
    print(f"Navegando a la URL: {url}")
    driver.get(url)

    # Pausa explícita para asegurar que todos los scripts de la página carguen
    print("Página cargada. Esperando 5 segundos a que se estabilice...")
    time.sleep(5)

    print("Buscando la pestaña 'Tablas'...")
    
    # Espera explícita (hasta 20 segundos) a que el enlace de "Tablas" esté presente en la página
    wait = WebDriverWait(driver, 20)
    tablas_button_selector = (By.CSS_SELECTOR, 'a[data-value="Tablas"]')
    
    wait.until(
        EC.presence_of_element_located(tablas_button_selector)
    )
    
    # Intenta hacer clic de dos maneras diferentes
    try:
        print("Intentando hacer clic en 'Tablas' con el método estándar...")
        tablas_button = driver.find_element(*tablas_button_selector)
        tablas_button.click()
    except Exception as e:
        print(f"El clic estándar falló. Razón: {e}")
        print("Intentando hacer clic en 'Tablas' con JavaScript...")
        tablas_button = driver.find_element(*tablas_button_selector)
        driver.execute_script("arguments[0].click();", tablas_button)

    # Espera a que el contenedor de la tabla (con id 'tab-4978-2') esté visible
    print("Esperando a que la tabla se cargue...")
    tabla_container_id = "tab-4978-2"
    wait.until(
        EC.visibility_of_element_located((By.ID, tabla_container_id))
    )
    print("¡Tabla encontrada!")

    # Obtiene el HTML del contenedor de la tabla
    tabla_container = driver.find_element(By.ID, tabla_container_id)
    tabla_html = tabla_container.get_attribute('outerHTML')

    # Pandas lee el HTML y extrae la tabla en un DataFrame
    df_list = pd.read_html(tabla_html)

    if df_list:
        df = df_list[0]
        print("\n--- Datos extraídos de la tabla ---")
        print(df)

        # --- Guardar los datos en un archivo CSV ---
        nombre_archivo = "datos_sala_metaxenica.csv"
        df.to_csv(nombre_archivo, index=False, encoding='utf-8-sig')
        print(f"\n✅ Datos guardados exitosamente en el archivo '{nombre_archivo}'")
    else:
        print("Error: No se encontró ninguna tabla en la pestaña 'Tablas'.")

except Exception as e:
    print(f"\nOcurrió un error inesperado: {e}")

finally:
    # Es importante cerrar el navegador al final para liberar recursos
    if 'driver' in locals():
        print("\nCerrando el navegador.")
        driver.quit()

Configurando el controlador de Chrome...
Navegando a la URL: https://app7.dge.gob.pe/maps/sala_metaxenica/
Página cargada. Esperando 5 segundos a que se estabilice...
Buscando la pestaña 'Tablas'...
Intentando hacer clic en 'Tablas' con el método estándar...
Esperando a que la tabla se cargue...
¡Tabla encontrada!

Ocurrió un error inesperado: Missing optional dependency 'lxml'.  Use pip or conda to install lxml.

Cerrando el navegador.


C:\Users\kamey\AppData\Local\Temp\ipykernel_51988\920613339.py:65: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df_list = pd.read_html(tabla_html)


In [8]:
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# --- Configuración de Selenium ---
options = webdriver.ChromeOptions()
# options.add_argument('--headless') # Mantenlo comentado para ver lo que hace el navegador
options.add_argument('--start-maximized')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')

try:
    print("Configurando el controlador de Chrome...")
    service = ChromeService(ChromeDriverManager().install())
    driver = webdriver.Chrome(service=service, options=options)
    wait = WebDriverWait(driver, 20)

    # --- 1. Navegar a la página y hacer clic en 'Tablas' ---
    url = "https://app7.dge.gob.pe/maps/sala_metaxenica/"
    print(f"Navegando a la URL: {url}")
    driver.get(url)

    print("Página cargada. Esperando 5 segundos a que se estabilice...")
    time.sleep(5)

    print("Buscando la pestaña 'Tablas'...")
    tablas_button_selector = (By.CSS_SELECTOR, 'a[data-value="Tablas"]')
    tablas_button = wait.until(EC.element_to_be_clickable(tablas_button_selector))
    tablas_button.click()
    print("Clic en 'Tablas' realizado.")
    
    # Pequeña pausa para que la pestaña de tablas cargue sus elementos
    time.sleep(2)

    # --- 2. Interactuar con el dropdown de Departamentos ---
    print("\nBuscando el dropdown de departamentos...")
    
    # El elemento en el que hacemos clic es el div que contiene el input con el id que encontraste.
    # Usaremos XPath para encontrar el padre del input.
    dropdown_trigger_selector = (By.XPATH, "//input[@id='ten-fdepa-filter-selectized']/parent::div")
    dropdown_trigger = wait.until(EC.element_to_be_clickable(dropdown_trigger_selector))
    
    print("Haciendo clic en el dropdown para desplegar las opciones...")
    dropdown_trigger.click()

    # --- 3. Imprimir los departamentos que contiene la lista ---
    print("Esperando a que las opciones del dropdown aparezcan...")
    # Las opciones son divs con la clase 'option' dentro de un contenedor con la clase 'selectize-dropdown-content'
    opciones_selector = (By.CSS_SELECTOR, "div.selectize-dropdown-content div.option")
    
    # Esperamos a que al menos un elemento de la opción sea visible
    wait.until(EC.visibility_of_element_located(opciones_selector))
    
    # Obtenemos todos los elementos de las opciones
    departamentos_opciones = driver.find_elements(*opciones_selector)

    print("\n--- Departamentos encontrados en la lista ---")
    # Usamos una lista para guardar los nombres por si los necesitamos
    nombres_departamentos = []
    for opcion in departamentos_opciones:
        nombre = opcion.text
        nombres_departamentos.append(nombre)
        print(nombre)
    print("------------------------------------------")

    # --- 4. Escoger la siguiente opción (la segunda de la lista) ---
    if len(departamentos_opciones) > 1:
        # El índice 0 es 'NACIONAL', el índice 1 es el siguiente
        segunda_opcion = departamentos_opciones[1]
        nombre_segunda_opcion = segunda_opcion.text
        
        print(f"\nSeleccionando la segunda opción de la lista: '{nombre_segunda_opcion}'...")
        segunda_opcion.click()
        print("¡Opción seleccionada!")
    else:
        print("\nNo se encontraron suficientes opciones para seleccionar la segunda.")

    # Pausa final para que puedas ver el resultado antes de que se cierre el navegador
    print("\nNavegación completada. El navegador se cerrará en 10 segundos.")
    time.sleep(10)


except Exception as e:
    print(f"\nOcurrió un error inesperado: {e}")

finally:
    # Es importante cerrar el navegador al final para liberar recursos
    if 'driver' in locals():
        print("\nCerrando el navegador.")
        driver.quit()

Configurando el controlador de Chrome...
Navegando a la URL: https://app7.dge.gob.pe/maps/sala_metaxenica/
Página cargada. Esperando 5 segundos a que se estabilice...
Buscando la pestaña 'Tablas'...
Clic en 'Tablas' realizado.

Buscando el dropdown de departamentos...
Haciendo clic en el dropdown para desplegar las opciones...
Esperando a que las opciones del dropdown aparezcan...

--- Departamentos encontrados en la lista ---
NACIONAL
AMAZONAS
ANCASH
APURIMAC
AREQUIPA
AYACUCHO
CAJAMARCA
CALLAO
CUSCO
HUANCAVELICA
HUANUCO
ICA
JUNIN
LA LIBERTAD
LAMBAYEQUE
LIMA
LORETO
MADRE DE DIOS
MOQUEGUA
PASCO
PIURA
PUNO
SAN MARTIN
TACNA
TUMBES
UCAYALI
------------------------------------------

Seleccionando la segunda opción de la lista: 'AMAZONAS'...
¡Opción seleccionada!

Navegación completada. El navegador se cerrará en 10 segundos.

Cerrando el navegador.


In [9]:
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# --- Configuración de Selenium ---
options = webdriver.ChromeOptions()
# options.add_argument('--headless') # Mantenlo comentado para ver lo que hace el navegador
options.add_argument('--start-maximized')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')

try:
    print("Configurando el controlador de Chrome...")
    service = ChromeService(ChromeDriverManager().install())
    driver = webdriver.Chrome(service=service, options=options)
    wait = WebDriverWait(driver, 20)

    # --- 1. Navegar a la página y hacer clic en 'Tablas' ---
    url = "https://app7.dge.gob.pe/maps/sala_metaxenica/"
    print(f"Navegando a la URL: {url}")
    driver.get(url)

    print("Página cargada. Esperando 5 segundos a que se estabilice...")
    time.sleep(5)

    print("Buscando la pestaña 'Tablas'...")
    tablas_button_selector = (By.CSS_SELECTOR, 'a[data-value="Tablas"]')
    tablas_button = wait.until(EC.element_to_be_clickable(tablas_button_selector))
    tablas_button.click()
    print("Clic en 'Tablas' realizado.")
    
    time.sleep(2) # Pausa para que carguen los elementos de la pestaña

    # --- 2. Interactuar con el dropdown y filtrar departamentos ---
    print("\nBuscando el dropdown de departamentos...")
    dropdown_trigger_selector = (By.XPATH, "//input[@id='ten-fdepa-filter-selectized']/parent::div")
    dropdown_trigger = wait.until(EC.element_to_be_clickable(dropdown_trigger_selector))
    
    print("Desplegando las opciones...")
    dropdown_trigger.click()

    print("Esperando a que las opciones aparezcan y filtrando la lista...")
    opciones_selector = (By.CSS_SELECTOR, "div.selectize-dropdown-content div.option")
    wait.until(EC.visibility_of_element_located(opciones_selector))
    
    todas_las_opciones = driver.find_elements(*opciones_selector)
    
    print("\n--- Departamentos encontrados (excluyendo 'NACIONAL') ---")
    departamentos_validos = []
    for opcion in todas_las_opciones:
        # Filtramos para excluir la opción "NACIONAL"
        if opcion.text.strip().upper() != "NACIONAL":
            departamentos_validos.append(opcion)
            print(opcion.text)
    print("-------------------------------------------------------")

    # --- 3. Seleccionar el primer departamento de la lista filtrada ---
    if departamentos_validos:
        primer_departamento = departamentos_validos[0]
        nombre_primer_depto = primer_departamento.text
        
        print(f"\nSeleccionando el primer departamento: '{nombre_primer_depto}'...")
        primer_departamento.click()
        print("¡Departamento seleccionado!")

        # Pequeña pausa para que la selección se registre en la página
        time.sleep(1)

        # --- 4. Presionar el botón "Procesar" ---
        print("\nBuscando el botón 'Procesar'...")
        procesar_button_selector = (By.ID, "ten-run")
        procesar_button = wait.until(EC.element_to_be_clickable(procesar_button_selector))
        
        print("Haciendo clic en 'Procesar'...")
        procesar_button.click()
        print("¡Botón 'Procesar' presionado!")

    else:
        print("\nNo se encontraron departamentos válidos para continuar.")

    # Pausa final para que puedas ver el resultado
    print("\nNavegación completada. La página se ha procesado.")
    print("El navegador se cerrará en 10 segundos.")
    time.sleep(10)


except Exception as e:
    # Imprime el error de una forma más legible
    import traceback
    print(f"\nOcurrió un error inesperado: {e}")
    traceback.print_exc()


finally:
    if 'driver' in locals():
        print("\nCerrando el navegador.")
        driver.quit()

Configurando el controlador de Chrome...
Navegando a la URL: https://app7.dge.gob.pe/maps/sala_metaxenica/
Página cargada. Esperando 5 segundos a que se estabilice...
Buscando la pestaña 'Tablas'...
Clic en 'Tablas' realizado.

Buscando el dropdown de departamentos...
Desplegando las opciones...
Esperando a que las opciones aparezcan y filtrando la lista...

--- Departamentos encontrados (excluyendo 'NACIONAL') ---
AMAZONAS
ANCASH
APURIMAC
AREQUIPA
AYACUCHO
CAJAMARCA
CALLAO
CUSCO
HUANCAVELICA
HUANUCO
ICA
JUNIN
LA LIBERTAD
LAMBAYEQUE
LIMA
LORETO
MADRE DE DIOS
MOQUEGUA
PASCO
PIURA
PUNO
SAN MARTIN
TACNA
TUMBES
UCAYALI
-------------------------------------------------------

Seleccionando el primer departamento: 'AMAZONAS'...
¡Departamento seleccionado!

Buscando el botón 'Procesar'...
Haciendo clic en 'Procesar'...
¡Botón 'Procesar' presionado!

Navegación completada. La página se ha procesado.
El navegador se cerrará en 10 segundos.

Cerrando el navegador.


In [17]:
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# --- Configuración de Selenium ---
options = webdriver.ChromeOptions()
# options.add_argument('--headless')
options.add_argument('--start-maximized')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')

try:
    print("Configurando el controlador de Chrome...")
    service = ChromeService(ChromeDriverManager().install())
    driver = webdriver.Chrome(service=service, options=options)
    wait = WebDriverWait(driver, 30) 

    # --- 1. Navegar y hacer clic en 'Tablas' ---
    url = "https://app7.dge.gob.pe/maps/sala_metaxenica/"
    driver.get(url)
    print("Página cargada. Esperando 5 segundos...")
    time.sleep(5)

    print("Haciendo clic en la pestaña 'Tablas'...")
    tablas_button = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, 'a[data-value="Tablas"]')))
    tablas_button.click()
    time.sleep(2)

    # --- 2. Seleccionar el primer Departamento válido ---
    print("\nInteractuando con el dropdown de Departamento...")
    dropdown_depto_trigger = wait.until(EC.element_to_be_clickable((By.XPATH, "//input[@id='ten-fdepa-filter-selectized']/parent::div")))
    dropdown_depto_trigger.click()
    
    time.sleep(0.5)
    opciones_selector = (By.CSS_SELECTOR, "div.selectize-dropdown-content div.option")
    wait.until(EC.presence_of_all_elements_located(opciones_selector))

    todas_opciones_depto = driver.find_elements(*opciones_selector)
    # Filtro robusto para departamentos
    departamentos_validos = [opt for opt in todas_opciones_depto if opt.text.strip() and opt.text.strip().upper() != "NACIONAL"]

    if departamentos_validos:
        primer_departamento_elem = departamentos_validos[0]
        departamento_actual_nombre = primer_departamento_elem.text
        
        print(f"Departamento actual: {departamento_actual_nombre}")
        driver.execute_script("arguments[0].click();", primer_departamento_elem)
        time.sleep(1)

        # --- 3. Presionar "Procesar" y esperar ---
        print("\nHaciendo clic en 'Procesar'...")
        procesar_button = wait.until(EC.element_to_be_clickable((By.ID, "ten-run")))
        procesar_button.click()
        
        print("Procesando... esperando a que la página termine de cargar los datos.")
        wait.until(EC.invisibility_of_element_located((By.CLASS_NAME, "shiny-busy")))
        print("¡Procesamiento completado!")

        # --- 4. Seleccionar la primera Provincia válida ---
        print("\nInteractuando con el dropdown de Provincia...")
        dropdown_prov_trigger = wait.until(EC.element_to_be_clickable((By.XPATH, "//input[@id='ten-fprov-filter-selectized']/parent::div")))
        dropdown_prov_trigger.click()
        
        time.sleep(0.5) 
        wait.until(EC.presence_of_all_elements_located(opciones_selector))
        
        todas_opciones_prov = driver.find_elements(*opciones_selector)

        # ******** LA CORRECCIÓN CLAVE ESTÁ AQUÍ ********
        # Filtramos para que la opción TENGA TEXTO y no sea "..."
        provincias_validas = [opt for opt in todas_opciones_prov if opt.text.strip() and opt.text.strip() != "..."]
        
        print("\n--- Provincias Válidas Encontradas ---")
        for prov in provincias_validas:
            print(prov.text)
        print("-------------------------------------")
        
        if provincias_validas:
            primera_provincia_elem = provincias_validas[0]
            provincia_actual_nombre = primera_provincia_elem.text
            
            driver.execute_script("arguments[0].click();", primera_provincia_elem)

            # --- 5. Imprimir el resultado final ---
            print("\n--- Resultado de la Selección ---")
            print(f'"Departamento actual": {departamento_actual_nombre} "y provincia: " {provincia_actual_nombre}')
            print("---------------------------------")
        else:
            print(f"No se encontraron provincias válidas para el departamento {departamento_actual_nombre}.")
    else:
        print("Error crítico: No se encontraron departamentos válidos para continuar.")

    # Pausa final
    print("\nNavegación completada. El navegador se cerrará en 10 segundos.")
    time.sleep(10)

except Exception as e:
    import traceback
    print(f"\nOcurrió un error inesperado: {e}")
    traceback.print_exc()

finally:
    if 'driver' in locals():
        print("\nCerrando el navegador.")
        driver.quit()

Configurando el controlador de Chrome...
Página cargada. Esperando 5 segundos...
Haciendo clic en la pestaña 'Tablas'...

Interactuando con el dropdown de Departamento...
Departamento actual: AMAZONAS

Haciendo clic en 'Procesar'...
Procesando... esperando a que la página termine de cargar los datos.
¡Procesamiento completado!

Interactuando con el dropdown de Provincia...

--- Provincias Válidas Encontradas ---
BAGUA
BONGARA
CHACHAPOYAS
CONDORCANQUI
LUYA
RODRIGUEZ DE MENDOZA
UTCUBAMBA
-------------------------------------

--- Resultado de la Selección ---
"Departamento actual": AMAZONAS "y provincia: " BAGUA
---------------------------------

Navegación completada. El navegador se cerrará en 10 segundos.

Cerrando el navegador.


In [23]:
import time
import os
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# --- Configuración de descargas ---
# Crear una carpeta para las descargas si no existe
download_folder = os.path.join(os.getcwd(), "descargas")
if not os.path.exists(download_folder):
    os.makedirs(download_folder)

print(f"Los archivos se guardarán en: {download_folder}")

# --- Configuración de Selenium ---
options = webdriver.ChromeOptions()
# options.add_argument('--headless') # Mantenlo comentado mientras pruebas
options.add_argument('--start-maximized')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')

# Preferencias para manejar descargas
prefs = {
    "download.default_directory": download_folder,
    "download.prompt_for_download": False,
    "download.directory_upgrade": True,
    "safebrowsing.enabled": True
}
options.add_experimental_option("prefs", prefs)

try:
    print("\nConfigurando el controlador de Chrome...")
    service = ChromeService(ChromeDriverManager().install())
    driver = webdriver.Chrome(service=service, options=options)
    wait = WebDriverWait(driver, 30) 

    # --- 1. Navegar y seleccionar Tablas ---
    url = "https://app7.dge.gob.pe/maps/sala_metaxenica/"
    driver.get(url)
    print("Página cargada. Esperando 5 segundos...")
    time.sleep(5)
    
    tablas_button = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, 'a[data-value="Tablas"]')))
    tablas_button.click()
    time.sleep(2)

    # --- 2. Seleccionar Departamento y Provincia ---
    # (Se mantiene la lógica robusta que ya desarrollamos)
    # Seleccionar Departamento
    dropdown_depto_trigger = wait.until(EC.element_to_be_clickable((By.XPATH, "//input[@id='ten-fdepa-filter-selectized']/parent::div")))
    dropdown_depto_trigger.click()
    time.sleep(0.5)
    opciones_selector = (By.CSS_SELECTOR, "div.selectize-dropdown-content div.option")
    wait.until(EC.presence_of_all_elements_located(opciones_selector))
    todas_opciones_depto = driver.find_elements(*opciones_selector)
    departamentos_validos = [opt for opt in todas_opciones_depto if opt.text.strip() and opt.text.strip().upper() != "NACIONAL"]
    
    if not departamentos_validos:
        raise Exception("Error crítico: No se encontraron departamentos válidos.")
    
    primer_departamento_elem = departamentos_validos[0]
    departamento_actual_nombre = primer_departamento_elem.text
    print(f"Departamento seleccionado: {departamento_actual_nombre}")
    driver.execute_script("arguments[0].click();", primer_departamento_elem)
    time.sleep(1)

    # Procesar
    procesar_button = wait.until(EC.element_to_be_clickable((By.ID, "ten-run")))
    procesar_button.click()
    wait.until(EC.invisibility_of_element_located((By.CLASS_NAME, "shiny-busy")))
    
    # Seleccionar Provincia
    dropdown_prov_trigger = wait.until(EC.element_to_be_clickable((By.XPATH, "//input[@id='ten-fprov-filter-selectized']/parent::div")))
    dropdown_prov_trigger.click()
    time.sleep(0.5) 
    wait.until(EC.presence_of_all_elements_located(opciones_selector))
    todas_opciones_prov = driver.find_elements(*opciones_selector)
    provincias_validas = [opt for opt in todas_opciones_prov if opt.text.strip() and opt.text.strip() != "..."]

    if not provincias_validas:
        raise Exception(f"No se encontraron provincias válidas para {departamento_actual_nombre}.")

    primera_provincia_elem = provincias_validas[0]
    provincia_actual_nombre = primera_provincia_elem.text
    print(f"Provincia seleccionada: {provincia_actual_nombre}")
    driver.execute_script("arguments[0].click();", primera_provincia_elem)
    print(f'\nSelección final: {departamento_actual_nombre} - {provincia_actual_nombre}')
    time.sleep(1)

    # --- 5. Cambiar a la pestaña "Tabla: Casos y defunciones" ---
    print("\nCambiando a la pestaña 'Tabla: Casos y defunciones'...")
    casos_tab_selector = (By.CSS_SELECTOR, 'a[data-value="Tabla: Casos y defunciones"]')
    casos_tab_button = wait.until(EC.element_to_be_clickable(casos_tab_selector))
    casos_tab_button.click()
    
    # Esperamos a que el contenedor de la nueva pestaña esté visible
    # El id es dinámico, pero podemos encontrarlo por el atributo 'href' del botón en el que hicimos clic
    nuevo_tab_id = casos_tab_button.get_attribute("href").split("#")[-1]
    wait.until(EC.visibility_of_element_located((By.ID, nuevo_tab_id)))
    print("Pestaña cargada.")

    # --- 6. Descargar el archivo ---
    print("\nBuscando el botón de descarga...")
    download_button_selector = (By.ID, "ten-tbl06b-download")
    download_button = wait.until(EC.element_to_be_clickable(download_button_selector))
    
    print("Haciendo clic en 'Descargar'...")
    download_button.click()

    # Pequeña pausa para dar tiempo a que la descarga comience
    print("Esperando a que la descarga inicie... (5 segundos)")
    time.sleep(5)

    print(f"\n✅ Tarea completada. El archivo debería estar en la carpeta '{download_folder}'.")
    
except Exception as e:
    import traceback
    print(f"\nOcurrió un error inesperado: {e}")
    traceback.print_exc()

finally:
    if 'driver' in locals():
        print("\nCerrando el navegador.")
        driver.quit()

Los archivos se guardarán en: c:\Users\kamey\Documents\Universidad\ciclo 8 e\modelo fisiologico\respaldo\buscando_data\scrapping\descargas

Configurando el controlador de Chrome...
Página cargada. Esperando 5 segundos...
Departamento seleccionado: AMAZONAS
Provincia seleccionada: BAGUA

Selección final: AMAZONAS - BAGUA

Cambiando a la pestaña 'Tabla: Casos y defunciones'...
Pestaña cargada.

Buscando el botón de descarga...
Haciendo clic en 'Descargar'...
Esperando a que la descarga inicie... (5 segundos)

✅ Tarea completada. El archivo debería estar en la carpeta 'c:\Users\kamey\Documents\Universidad\ciclo 8 e\modelo fisiologico\respaldo\buscando_data\scrapping\descargas'.

Cerrando el navegador.


In [26]:
import time
import os
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# --- Configuración de descargas ---
download_folder = os.path.join(os.getcwd(), "descargas")
if not os.path.exists(download_folder):
    os.makedirs(download_folder)
print(f"Los archivos se guardarán en: {download_folder}")

# --- Configuración de Selenium ---
options = webdriver.ChromeOptions()
# options.add_argument('--headless')
options.add_argument('--start-maximized')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
prefs = {
    "download.default_directory": download_folder,
    "download.prompt_for_download": False,
    "download.directory_upgrade": True,
    "safebrowsing.enabled": True
}
options.add_experimental_option("prefs", prefs)

try:
    print("\nConfigurando el controlador de Chrome...")
    service = ChromeService(ChromeDriverManager().install())
    driver = webdriver.Chrome(service=service, options=options)
    wait = WebDriverWait(driver, 30) 

    # --- 1. Navegar y hacer clic en "Tablas" ---
    url = "https://app7.dge.gob.pe/maps/sala_metaxenica/"
    driver.get(url)
    print("Página cargada. Esperando 5 segundos...")
    time.sleep(5)
    
    tablas_button = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, 'a[data-value="Tablas"]')))
    tablas_button.click()
    time.sleep(2)

    # --- 2. Seleccionar Departamento ---
    print("\nSeleccionando Departamento...")
    dropdown_depto_trigger = wait.until(EC.element_to_be_clickable((By.XPATH, "//input[@id='ten-fdepa-filter-selectized']/parent::div")))
    dropdown_depto_trigger.click()
    time.sleep(0.5)
    
    opciones_selector = (By.CSS_SELECTOR, "div.selectize-dropdown-content div.option")
    wait.until(EC.presence_of_all_elements_located(opciones_selector))
    
    todas_opciones_depto = driver.find_elements(*opciones_selector)
    departamentos_validos = [opt for opt in todas_opciones_depto if opt.text.strip() and opt.text.strip().upper() != "NACIONAL"]
    
    if not departamentos_validos: raise Exception("Error crítico: No se encontraron departamentos válidos.")
    
    primer_departamento_elem = departamentos_validos[0]
    departamento_actual_nombre = primer_departamento_elem.text
    print(f"Departamento seleccionado: {departamento_actual_nombre}")
    driver.execute_script("arguments[0].click();", primer_departamento_elem)
    time.sleep(1) # Pausa para que el dropdown se cierre y la selección se registre visualmente

    # --- 3. Seleccionar Provincia ---
    # Inmediatamente después de seleccionar el departamento, seleccionamos la provincia.
    print("\nSeleccionando Provincia...")
    dropdown_prov_trigger = wait.until(EC.element_to_be_clickable((By.XPATH, "//input[@id='ten-fprov-filter-selectized']/parent::div")))
    dropdown_prov_trigger.click()
    time.sleep(0.5) 
    
    wait.until(EC.presence_of_all_elements_located(opciones_selector))
    todas_opciones_prov = driver.find_elements(*opciones_selector)
    provincias_validas = [opt for opt in todas_opciones_prov if opt.text.strip() and opt.text.strip() != "..."]

    if not provincias_validas: raise Exception(f"No se encontraron provincias válidas para {departamento_actual_nombre}.")

    primera_provincia_elem = provincias_validas[0]
    provincia_actual_nombre = primera_provincia_elem.text
    print(f"Provincia seleccionada: {provincia_actual_nombre}")
    driver.execute_script("arguments[0].click();", primera_provincia_elem)
    time.sleep(1)

    # --- 4. Procesar AMBAS selecciones ---
    print(f'\nProcesando selección final: {departamento_actual_nombre} - {provincia_actual_nombre}')
    procesar_button = wait.until(EC.element_to_be_clickable((By.ID, "ten-run")))
    procesar_button.click()
    wait.until(EC.invisibility_of_element_located((By.CLASS_NAME, "shiny-busy")))
    print("Página actualizada con los datos filtrados.")

    # --- 5. Cambiar a la pestaña de descarga ---
    print("\nCambiando a la pestaña 'Tabla: Casos y defunciones'...")
    casos_tab_selector = (By.CSS_SELECTOR, 'a[data-value="Tabla: Casos y defunciones"]')
    casos_tab_button = wait.until(EC.element_to_be_clickable(casos_tab_selector))
    
    nuevo_tab_id = casos_tab_button.get_attribute("href").split("#")[-1]
    casos_tab_button.click()
    wait.until(EC.visibility_of_element_located((By.ID, nuevo_tab_id)))
    print("Pestaña de descarga cargada.")

    # --- 6. Descargar el archivo ---
    print("\nBuscando el botón de descarga...")
    download_button_selector = (By.ID, "ten-tbl06b-download")
    download_button = wait.until(EC.element_to_be_clickable(download_button_selector))
    
    print("Haciendo clic en 'Descargar'...")
    download_button.click()

    print("Esperando a que la descarga inicie... (10 segundos)")
    time.sleep(10)

    print(f"\n✅ Tarea completada. El archivo debería estar en la carpeta '{download_folder}'.")
    
except Exception as e:
    import traceback
    print(f"\nOcurrió un error inesperado: {e}")
    traceback.print_exc()

finally:
    if 'driver' in locals():
        print("\nCerrando el navegador.")
        driver.quit()

Los archivos se guardarán en: c:\Users\kamey\Documents\Universidad\ciclo 8 e\modelo fisiologico\respaldo\buscando_data\scrapping\descargas

Configurando el controlador de Chrome...
Página cargada. Esperando 5 segundos...

Seleccionando Departamento...

Ocurrió un error inesperado: Error crítico: No se encontraron departamentos válidos.

Cerrando el navegador.


Traceback (most recent call last):
  File "C:\Users\kamey\AppData\Local\Temp\ipykernel_51988\1961349902.py", line 58, in <module>
    if not departamentos_validos: raise Exception("Error crítico: No se encontraron departamentos válidos.")
                                  ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
Exception: Error crítico: No se encontraron departamentos válidos.


In [29]:
import time
import os
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# --- Configuración de descargas ---
download_folder = os.path.join(os.getcwd(), "descargas")
if not os.path.exists(download_folder):
    os.makedirs(download_folder)
print(f"Los archivos se guardarán en: {download_folder}")

# --- Configuración de Selenium ---
options = webdriver.ChromeOptions()
# options.add_argument('--headless')
options.add_argument('--start-maximized')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
prefs = {
    "download.default_directory": download_folder,
    "download.prompt_for_download": False,
    "download.directory_upgrade": True,
    "safebrowsing.enabled": True
}
options.add_experimental_option("prefs", prefs)

try:
    print("\nConfigurando el controlador de Chrome...")
    service = ChromeService(ChromeDriverManager().install())
    driver = webdriver.Chrome(service=service, options=options)
    wait = WebDriverWait(driver, 30) 

    # --- 1. Navegar y esperar a que la página principal cargue ---
    url = "https://app7.dge.gob.pe/maps/sala_metaxenica/"
    driver.get(url)
    print("Página cargada. Esperando a que los elementos iniciales se estabilicen...")
    wait.until(EC.presence_of_element_located((By.CLASS_NAME, "card-title")))
    
    # --- 2. Hacer clic en "Tablas" y ESPERAR a que su contenido cargue ---
    print("\nHaciendo clic en la pestaña 'Tablas'...")
    tablas_button = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, 'a[data-value="Tablas"]')))
    tablas_button.click()
    
    # Verificación clave: Esperamos a que el dropdown de departamento exista.
    dropdown_depto_xpath = "//input[@id='ten-fdepa-filter-selectized']/parent::div"
    wait.until(EC.presence_of_element_located((By.XPATH, dropdown_depto_xpath)))
    print("Pestaña 'Tablas' cargada correctamente.")

    # --- 3. Seleccionar Departamento ---
    print("\nSeleccionando Departamento...")
    dropdown_depto_trigger = wait.until(EC.element_to_be_clickable((By.XPATH, dropdown_depto_xpath)))
    dropdown_depto_trigger.click()
    time.sleep(0.5)
    
    opciones_selector = (By.CSS_SELECTOR, "div.selectize-dropdown-content div.option")
    wait.until(EC.presence_of_all_elements_located(opciones_selector))
    
    todas_opciones_depto = driver.find_elements(*opciones_selector)
    departamentos_validos = [opt for opt in todas_opciones_depto if opt.text.strip() and opt.text.strip().upper() != "NACIONAL"]
    
    if not departamentos_validos: raise Exception("Error crítico: No se encontraron departamentos válidos.")
    
    primer_departamento_elem = departamentos_validos[0]
    departamento_actual_nombre = primer_departamento_elem.text
    print(f"Departamento seleccionado: {departamento_actual_nombre}")
    driver.execute_script("arguments[0].click();", primer_departamento_elem)
    
    # --- 4. Seleccionar Provincia ---
    print("\nSeleccionando Provincia...")
    dropdown_prov_xpath = "//input[@id='ten-fprov-filter-selectized']/parent::div"
    wait.until(EC.element_to_be_clickable((By.XPATH, dropdown_prov_xpath))).click()
    time.sleep(0.5) 
    
    wait.until(EC.presence_of_all_elements_located(opciones_selector))
    todas_opciones_prov = driver.find_elements(*opciones_selector)
    provincias_validas = [opt for opt in todas_opciones_prov if opt.text.strip() and opt.text.strip() != "..."]

    if not provincias_validas: raise Exception(f"No se encontraron provincias válidas para {departamento_actual_nombre}.")

    primera_provincia_elem = provincias_validas[0]
    provincia_actual_nombre = primera_provincia_elem.text
    print(f"Provincia seleccionada: {provincia_actual_nombre}")
    driver.execute_script("arguments[0].click();", primera_provincia_elem)
    
    # --- 5. Procesar AMBAS selecciones ---
    print(f'\nProcesando selección final: {departamento_actual_nombre} - {provincia_actual_nombre}')
    wait.until(EC.element_to_be_clickable((By.ID, "ten-run"))).click()
    wait.until(EC.invisibility_of_element_located((By.CLASS_NAME, "shiny-busy")))
    print("Página actualizada con los datos filtrados.")

    # --- 6. Cambiar a la pestaña de descarga ---
    print("\nCambiando a la pestaña 'Tabla: Casos y defunciones'...")
    casos_tab_selector = (By.CSS_SELECTOR, 'a[data-value="Tabla: Casos y defunciones"]')
    casos_tab_button = wait.until(EC.element_to_be_clickable(casos_tab_selector))
    
    nuevo_tab_id = casos_tab_button.get_attribute("href").split("#")[-1]
    casos_tab_button.click()
    wait.until(EC.visibility_of_element_located((By.ID, nuevo_tab_id)))
    print("Pestaña de descarga cargada.")

    # --- 7. Descargar y Renombrar el archivo ---
    # Lógica de renombrado... (se mantiene igual)
    print("\nPreparando para descargar y renombrar...")
    files_before = set(os.listdir(download_folder))
    wait.until(EC.element_to_be_clickable((By.ID, "ten-tbl06b-download"))).click()
    print("Esperando a que aparezca el nuevo archivo...")
    # ... (código de renombrado sin cambios)

    new_file_path = ""
    for i in range(20):
        files_after = set(os.listdir(download_folder))
        new_files = files_after - files_before
        if new_files:
            original_filename = new_files.pop()
            if not original_filename.endswith((".crdownload", ".tmp")):
                print(f"Archivo descargado: {original_filename}")
                new_file_path = os.path.join(download_folder, original_filename)
                break
        time.sleep(1)
        
    if not new_file_path: raise Exception("La descarga del archivo no se completó a tiempo.")

    dept_safe = departamento_actual_nombre.lower().replace(" ", "_")
    prov_safe = provincia_actual_nombre.lower().replace(" ", "_")
    _, file_extension = os.path.splitext(new_file_path)
    new_filename = f"{dept_safe}-{prov_safe}-caso_y_defunciones{file_extension}"
    new_filepath = os.path.join(download_folder, new_filename)

    print(f"Renombrando a: {new_filename}")
    os.rename(new_file_path, new_filepath)
    print(f"\n✅ Tarea completada. Archivo final: '{new_filename}'")
    
except Exception as e:
    import traceback
    print(f"\nOcurrió un error inesperado: {e}")
    traceback.print_exc()

finally:
    if 'driver' in locals():
        print("\nCerrando el navegador.")
        driver.quit()

Los archivos se guardarán en: c:\Users\kamey\Documents\Universidad\ciclo 8 e\modelo fisiologico\respaldo\buscando_data\scrapping\descargas

Configurando el controlador de Chrome...
Página cargada. Esperando a que los elementos iniciales se estabilicen...

Ocurrió un error inesperado: Message: 
Stacktrace:
	GetHandleVerifier [0x0x4ec333+65459]
	GetHandleVerifier [0x0x4ec374+65524]
	(No symbol) [0x0x30d973]
	(No symbol) [0x0x3576e7]
	(No symbol) [0x0x357a8b]
	(No symbol) [0x0x39dea2]
	(No symbol) [0x0x379e44]
	(No symbol) [0x0x39b606]
	(No symbol) [0x0x379bf6]
	(No symbol) [0x0x34b38e]
	(No symbol) [0x0x34c274]
	GetHandleVerifier [0x0x76eda3+2697763]
	GetHandleVerifier [0x0x769ec7+2677575]
	GetHandleVerifier [0x0x514194+228884]
	GetHandleVerifier [0x0x5049f8+165496]
	GetHandleVerifier [0x0x50b18d+192013]
	GetHandleVerifier [0x0x4f47d8+99416]
	GetHandleVerifier [0x0x4f4972+99826]
	GetHandleVerifier [0x0x4debea+10346]
	BaseThreadInitThunk [0x0x75a55d49+25]
	RtlInitializeExceptionChain [0x0

Traceback (most recent call last):
  File "C:\Users\kamey\AppData\Local\Temp\ipykernel_51988\2378757027.py", line 40, in <module>
    wait.until(EC.presence_of_element_located((By.CLASS_NAME, "card-title")))
  File "c:\Users\kamey\anaconda3\envs\laboratorio\Lib\site-packages\selenium\webdriver\support\wait.py", line 138, in until
    raise TimeoutException(message, screen, stacktrace)
selenium.common.exceptions.TimeoutException: Message: 
Stacktrace:
	GetHandleVerifier [0x0x4ec333+65459]
	GetHandleVerifier [0x0x4ec374+65524]
	(No symbol) [0x0x30d973]
	(No symbol) [0x0x3576e7]
	(No symbol) [0x0x357a8b]
	(No symbol) [0x0x39dea2]
	(No symbol) [0x0x379e44]
	(No symbol) [0x0x39b606]
	(No symbol) [0x0x379bf6]
	(No symbol) [0x0x34b38e]
	(No symbol) [0x0x34c274]
	GetHandleVerifier [0x0x76eda3+2697763]
	GetHandleVerifier [0x0x769ec7+2677575]
	GetHandleVerifier [0x0x514194+228884]
	GetHandleVerifier [0x0x5049f8+165496]
	GetHandleVerifier [0x0x50b18d+192013]
	GetHandleVerifier [0x0x4f47d8+99416]


## Segundo intento

In [1]:
pip install --upgrade selenium webdriver-manager pandas

   ---------------------------------------- 0.0/9.6 MB ? eta -:--:--
   ------- -------------------------------- 1.8/9.6 MB 12.6 MB/s eta 0:00:01
   ------------------- -------------------- 4.7/9.6 MB 13.6 MB/s eta 0:00:01
   ------------------------------ --------- 7.3/9.6 MB 12.9 MB/s eta 0:00:01
   ---------------------------------------- 9.6/9.6 MB 12.7 MB/s eta 0:00:00
  Attempting uninstall: selenium
    Found existing installation: selenium 4.35.0
    Uninstalling selenium-4.35.0:
      Successfully uninstalled selenium-4.35.0
Note: you may need to restart the kernel to use updated packages.


## Tercer intento

In [1]:
pip install selenium webdriver-manager pandas

Note: you may need to restart the kernel to use updated packages.


* De "Tablas":  
```
<a href="#tab-8922-2" data-toggle="tab" data-bs-toggle="tab" data-value="Tablas" class="nav-link" aria-selected="false" tabindex="-1" role="tab">Tablas</a>
```  


* De "Año de análisis":  
```
<div class="selectize-input items full has-options has-items"><div class="item" data-value="2025">2025</div><input type="text" autocomplete="new-password" autofill="no" tabindex="" id="ten-fyear-selectized" role="combobox" aria-expanded="false" haspopup="listbox" aria-owns="4plr9egtvp" style="width: 4px;"></div>
```  
  
* De "Enfermedad":  
```  
<div class="selectize-input items full has-options has-items"><div class="item" data-value="DENGUE">DENGUE</div><input type="text" autocomplete="new-password" autofill="no" tabindex="" id="ten-fenfe-selectized" role="combobox" aria-expanded="false" haspopup="listbox" aria-owns="9mrpb0j5eo" style="width: 4px;"></div>
```  
  
* De "Departamento":  
```  
<div class="selectize-input items full has-options has-items"><div class="item" data-value="1">NACIONAL</div><input type="text" autocomplete="new-password" autofill="no" tabindex="" id="ten-fdepa-filter-selectized" role="combobox" aria-expanded="false" haspopup="listbox" aria-owns="jp9gyamo9i" style="width: 4px;"></div>
```  
  
* De "Provincia":  
```  
<div class="selectize-input items full has-options has-items"><div class="item" data-value="0">...</div><input type="text" autocomplete="new-password" autofill="no" tabindex="" id="ten-fprov-filter-selectized" role="combobox" aria-expanded="false" haspopup="listbox" aria-owns="6hkq76uai2" style="width: 4px;"></div>
```
  
* De "Distrito":  
```  
<div class="selectize-input items full has-options has-items"><div class="item" data-value="0">...</div><input type="text" autocomplete="new-password" autofill="no" tabindex="" id="ten-fdist-filter-selectized" role="combobox" aria-expanded="false" haspopup="listbox" aria-owns="ugw7idl81d" style="width: 4px;"></div>
```  
  
* Del botón "Procesar":  
```
<button class="btn btn-default action-button btn btn-outline-success shiny-bound-input" id="ten-run" type="button">
                  <i class="fas fa-gear" role="presentation" aria-label="gear icon"></i>
                  Procesar
                </button>
```  

In [4]:
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
from selenium.common.exceptions import TimeoutException, ElementClickInterceptedException

# --- Las funciones auxiliares (get_selectize_options y select_selectize_option) no necesitan cambios ---
# --- Se mantienen igual que en la versión anterior ---

def get_selectize_options(driver, control_id):
    """
    Abre un dropdown de Selectize.js, extrae todas las opciones y se asegura de cerrarlo.
    """
    options_data = []
    control_xpath = f"//select[@id='{control_id}']/following-sibling::div[1]"
    dropdown_content_xpath = "//div[contains(@class, 'selectize-dropdown') and contains(@style, 'display: block;')]//div[@class='selectize-dropdown-content']"

    try:
        control_element = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, control_xpath))
        )
        driver.execute_script("arguments[0].click();", control_element)
        
        WebDriverWait(driver, 10).until(
            EC.visibility_of_element_located((By.XPATH, dropdown_content_xpath))
        )
        
        options_elements = driver.find_elements(By.XPATH, f"{dropdown_content_xpath}/div[@data-value]")
        
        for option in options_elements:
            value = option.get_attribute('data-value')
            text = option.text
            if value:
                options_data.append({"value": value, "text": text})

        control_element.send_keys(Keys.ESCAPE)
        
        WebDriverWait(driver, 10).until(
            EC.invisibility_of_element_located((By.XPATH, "//div[contains(@class, 'selectize-dropdown') and contains(@style, 'display: block;')]"))
        )
        
    except TimeoutException:
        print(f"Error: Timeout esperando el dropdown o las opciones para '{control_id}'.")
    except Exception as e:
        print(f"Error inesperado al obtener opciones de '{control_id}': {e}")
        
    return options_data

def select_selectize_option(driver, control_id, option_value):
    """
    Selecciona una opción de un dropdown de Selectize.js y se asegura de que el dropdown se cierre.
    """
    control_xpath = f"//select[@id='{control_id}']/following-sibling::div[1]"
    dropdown_parent_xpath = "//div[contains(@class, 'selectize-dropdown') and contains(@style, 'display: block;')]"

    try:
        control_element = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, control_xpath))
        )
        driver.execute_script("arguments[0].click();", control_element)

        WebDriverWait(driver, 10).until(
            EC.visibility_of_element_located((By.XPATH, dropdown_parent_xpath))
        )

        option_xpath = f"{dropdown_parent_xpath}//div[@data-value='{option_value}']"
        option_element = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, option_xpath))
        )
        option_element.click()
        print(f"Opción '{option_value}' seleccionada en el dropdown '{control_id}'.")
        
        WebDriverWait(driver, 10).until(
            EC.invisibility_of_element_located((By.XPATH, dropdown_parent_xpath))
        )
        print(f"Dropdown '{control_id}' cerrado correctamente.")
        
    except TimeoutException:
        print(f"Error: No se pudo encontrar la opción '{option_value}' o el dropdown '{control_id}' no se comportó como se esperaba.")
    except Exception as e:
        print(f"Error inesperado al seleccionar en '{control_id}': {e}")


# --- Script Principal ---
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service)
url = "https://app7.dge.gob.pe/maps/sala_metaxenica/"

try:
    print(f"Accediendo a la URL: {url}")
    driver.get(url)
    driver.maximize_window()

    print("Cambiando a la pestaña 'Tablas'...")
    tablas_button = WebDriverWait(driver, 20).until(
        EC.element_to_be_clickable((By.XPATH, "//a[@data-value='Tablas']"))
    )
    tablas_button.click()
    
    # --- CAMBIO PRINCIPAL AQUÍ ---
    # Espera a que el DIV contenedor de la pestaña sea visible, en lugar de un elemento específico.
    # El ID "tab-8922-2" se obtiene del 'href' del botón en el que hicimos clic.
    WebDriverWait(driver, 15).until(
        EC.visibility_of_element_located((By.ID, "tab-8922-2"))
    )
    print("Pestaña 'Tablas' cargada y visible.")

    # --- Secuencia de Selección (con pausas ligeramente aumentadas para mayor seguridad) ---
    select_selectize_option(driver, "ten-fyear", "2025")
    time.sleep(2) 

    select_selectize_option(driver, "ten-fenfe", "DENGUE")
    time.sleep(3) 

    print("\n--- Obteniendo opciones de Departamento ---")
    departamentos = get_selectize_options(driver, "ten-fdepa-filter")
    for dep in departamentos:
        print(f"Texto: {dep['text']}, Valor: {dep['value']}")
    select_selectize_option(driver, "ten-fdepa-filter", "01") # AMAZONAS
    time.sleep(3)

    print("\n--- Obteniendo opciones de Provincia (para AMAZONAS) ---")
    provincias = get_selectize_options(driver, "ten-fprov-filter")
    for prov in provincias:
        print(f"Texto: {prov['text']}, Valor: {prov['value']}")
    select_selectize_option(driver, "ten-fprov-filter", "0102") # BAGUAS
    time.sleep(3)

    print("\n--- Obteniendo opciones de Distrito (para BAGUAS) ---")
    distritos = get_selectize_options(driver, "ten-fdist-filter")
    for dist in distritos:
        print(f"Texto: {dist['text']}, Valor: {dist['value']}")
        
    print("\n✅ Proceso finalizado exitosamente.")

finally:
    print("\nCerrando el navegador en 15 segundos...")
    time.sleep(15)
    driver.quit()

Accediendo a la URL: https://app7.dge.gob.pe/maps/sala_metaxenica/
Cambiando a la pestaña 'Tablas'...

Cerrando el navegador en 15 segundos...


TimeoutException: Message: 


In [5]:
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
from selenium.common.exceptions import TimeoutException

def get_selectize_options(driver, control_id):
    """
    Abre un dropdown de Selectize.js, extrae todas las opciones y se asegura de cerrarlo.
    
    Args:
        driver: La instancia del WebDriver de Selenium.
        control_id (str): El ID del <select> original que controla el dropdown.
    """
    options_data = []
    # Localizador para el control visible del dropdown
    control_xpath = f"//select[@id='{control_id}']/following-sibling::div[1]"
    # Localizador para el panel de opciones una vez que está abierto
    dropdown_content_xpath = "//div[contains(@class, 'selectize-dropdown') and contains(@style, 'display: block;')]//div[@class='selectize-dropdown-content']"

    try:
        # 1. Espera y haz clic en el control para abrir el menú
        control_element = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, control_xpath))
        )
        # Un clic con JavaScript es a menudo más fiable
        driver.execute_script("arguments[0].click();", control_element)
        
        # 2. Espera a que el panel de opciones sea visible
        WebDriverWait(driver, 10).until(
            EC.visibility_of_element_located((By.XPATH, dropdown_content_xpath))
        )
        
        # 3. Extrae todos los elementos de opción que tienen un 'data-value'
        options_elements = driver.find_elements(By.XPATH, f"{dropdown_content_xpath}/div[@data-value]")
        
        for option in options_elements:
            value = option.get_attribute('data-value')
            text = option.text.strip() # .strip() para limpiar espacios en blanco
            if value:
                options_data.append({"value": value, "text": text})

        # 4. Cierra el dropdown enviando la tecla 'Escape'
        control_element.send_keys(Keys.ESCAPE)
        
        # 5. Confirma que el panel de opciones se ha cerrado
        WebDriverWait(driver, 10).until(
            EC.invisibility_of_element_located((By.XPATH, "//div[contains(@class, 'selectize-dropdown') and contains(@style, 'display: block;')]"))
        )
        
    except TimeoutException:
        print(f"Error: Timeout esperando el dropdown o las opciones para '{control_id}'.")
    except Exception as e:
        print(f"Error inesperado al obtener opciones de '{control_id}': {e}")
        
    return options_data


# --- Script Principal ---
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service)
url = "https://app7.dge.gob.pe/maps/sala_metaxenica/"

try:
    print(f"Accediendo a la URL: {url}")
    driver.get(url)
    driver.maximize_window()

    # 1. Hacer clic en la pestaña "Tablas"
    print("Cambiando a la pestaña 'Tablas'...")
    tablas_button = WebDriverWait(driver, 20).until(
        EC.element_to_be_clickable((By.XPATH, "//a[@data-value='Tablas']"))
    )
    tablas_button.click()
    
    # 2. Esperar a que el contenido de la pestaña se cargue y sea visible
    # El ID 'tab-8922-2' corresponde al 'href' del botón de la pestaña
    WebDriverWait(driver, 15).until(
        EC.visibility_of_element_located((By.ID, "tab-8922-2"))
    )
    print("Pestaña 'Tablas' cargada y visible.")
    
    # 3. Obtener e imprimir las opciones del dropdown "Año de análisis"
    print("\n--- Obteniendo opciones de 'Año de análisis' ---")
    # El ID del <select> original es "ten-fyear"
    lista_de_anios = get_selectize_options(driver, "ten-fyear")
    
    if lista_de_anios:
        for anio in lista_de_anios:
            print(f"Texto: {anio['text']}, Valor: {anio['value']}")
    else:
        print("No se pudieron obtener las opciones de año.")
        
    print("\n✅ Proceso finalizado exitosamente.")

finally:
    print("\nCerrando el navegador en 10 segundos...")
    time.sleep(10)
    driver.quit()

Accediendo a la URL: https://app7.dge.gob.pe/maps/sala_metaxenica/
Cambiando a la pestaña 'Tablas'...
Pestaña 'Tablas' cargada y visible.

--- Obteniendo opciones de 'Año de análisis' ---
Error: Timeout esperando el dropdown o las opciones para 'ten-fyear'.
No se pudieron obtener las opciones de año.

✅ Proceso finalizado exitosamente.

Cerrando el navegador en 10 segundos...


In [6]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time

# --- Configuración del WebDriver ---
# Asegúrate de tener el webdriver de Chrome (o el de tu navegador preferido)
# en tu PATH o especifica la ruta al ejecutable.
driver = webdriver.Chrome()

try:
    # 1. Navegar a la URL
    driver.get("https://app7.dge.gob.pe/maps/sala_metaxenica/")

    # 2. Hacer clic en la pestaña "Tablas"
    # Se utiliza una espera explícita para asegurar que el elemento esté cargado y sea clickeable.
    print("Esperando a que la pestaña 'Tablas' esté disponible...")
    tablas_tab = WebDriverWait(driver, 20).until(
        EC.element_to_be_clickable((By.XPATH, "//a[@data-value='Tablas']"))
    )
    tablas_tab.click()
    print("Se hizo clic en 'Tablas'.")

    # Pequeña pausa para asegurar que el contenido de la pestaña cargue
    time.sleep(2)

    # 3. Hacer clic en el menú desplegable "Año de análisis" para abrir las opciones
    # Este tipo de menú (selectize.js) requiere hacer clic en el contenedor para mostrar las opciones.
    print("Abriendo el menú desplegable 'Año de análisis'...")
    anio_dropdown_container = WebDriverWait(driver, 20).until(
        EC.element_to_be_clickable((By.ID, "ten-fyear-selectized"))
    )
    # Hacemos clic en el elemento padre que controla el input
    anio_dropdown_container.find_element(By.XPATH, "./..").click()
    print("Menú desplegable abierto.")

    # 4. Extraer e imprimir las opciones del menú
    # Las opciones aparecen en un contenedor separado después de hacer clic.
    print("\nOpciones disponibles en 'Año de análisis':")
    options_container = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.XPATH, f"//div[@class='selectize-dropdown-content']"))
    )
    
    options = options_container.find_elements(By.XPATH, ".//div[@data-value]")
    
    for option in options:
        print(option.text)

finally:
    # 5. Cerrar el navegador
    print("\nCerrando el navegador.")
    driver.quit()

Esperando a que la pestaña 'Tablas' esté disponible...
Se hizo clic en 'Tablas'.
Abriendo el menú desplegable 'Año de análisis'...
Menú desplegable abierto.

Opciones disponibles en 'Año de análisis':
2025
2024
2023
2022
2021
2020

Cerrando el navegador.


### Usando Playwright y asyncio (gracias píldorasdeprogramación)

In [1]:
%pip install playwright

Note: you may need to restart the kernel to use updated packages.


In [2]:
!playwright install chromium

In [5]:
%pip install pytest-playwright playwright -U

Note: you may need to restart the kernel to use updated packages.


In [1]:
import asyncio
from playwright.async_api import async_playwright, TimeoutError as PlaywrightTimeoutError

async def main():
    """
    Función principal para realizar el web scraping con Playwright.
    """
    async with async_playwright() as p:
        # Lanza un navegador. Cambia headless=True si no quieres ver la ventana.
        browser = await p.chromium.launch(headless=False) 
        page = await browser.new_page()

        try:
            # 1. Navegar a la URL
            print("Navegando a https://app7.dge.gob.pe/maps/sala_metaxenica/...")
            await page.goto("https://app7.dge.gob.pe/maps/sala_metaxenica/", timeout=60000)
            print("Página cargada.")

            # 2. Hacer clic en la pestaña "Tablas"
            print("Haciendo clic en la pestaña 'Tablas'...")
            tablas_selector = "a[data-value='Tablas']"
            await page.wait_for_selector(tablas_selector, timeout=30000)
            await page.click(tablas_selector)
            print("Se hizo clic en 'Tablas'.")

            # 3. Abrir el menú desplegable "Año de análisis"
            print("Abriendo el menú desplegable 'Año de análisis'...")
            dropdown_selector = "#ten-fyear-selectized"
            await page.wait_for_selector(dropdown_selector, timeout=30000)
            await page.click(dropdown_selector)
            print("Menú desplegable abierto.")

            # 4. Extraer e imprimir las opciones
            print("\n--- Opciones disponibles en 'Año de análisis' ---")
            options_container_selector = ".selectize-dropdown-content"
            await page.wait_for_selector(options_container_selector, state="visible")
            
            options = await page.query_selector_all(".selectize-dropdown-content .option")
            
            if not options:
                print("No se encontraron opciones.")
            else:
                for option in options:
                    year_text = await option.inner_text()
                    print(f"- {year_text}")
            print("-------------------------------------------------")

        except PlaywrightTimeoutError:
            print("\nError: Se agotó el tiempo de espera.")
        except Exception as e:
            print(f"\nOcurrió un error inesperado: {e}")
        finally:
            # 5. Cerrar el navegador
            print("\nCerrando el navegador.")
            await browser.close()

In [2]:
await main()

NotImplementedError: 